#Defining class `Var`

In [ ]:
import math

class Var:
    def __init__(self, value):
        self.value = value
        self.children = []
        self.grad_value = None  #Initialize to None, which means it's not yet evaluated

    def grad(self):
        #recurse only if the value is not yet cached
        if self.grad_value is None:
            #calculate derivative using chaing rule
            self.grad_value = sum(weight * var.grad()
                                  for weight, var in self.children)
        return self.grad_value
    
    def __str__(self):
        return str(self.value)

    def __mul__(self, other): # z=x*y    dz/dx=y  dz/dy=x
        z = Var(self.value * other.value)
        self.children.append((other.value, z)) #x.children.append((y.value,z))  x.child =[dz/dx=y, z] <--assign children z to x
        other.children.append((self.value, z))  #y.children.append((x.value,z)) y.child =[dz/dy=x,z] <-- assign children z to y
        return z

    def __add__(self, other): #z=x+y, dz/dx=1, dz/dy=1
        z=Var(self.value+other.value)
        self.children.append((1.0,z)) #x.child =[dz/dx=1, z]
        other.children.append((1.0,z)) #y.child =[dz/dy=1, z]
        return z

    def __sub__(self, other): #z=x-y, dz/dx=1, dz/dy=1
        z=Var(self.value-other.value)
        self.children.append((1.0,z)) #x.child =[dz/dx=1, z]
        other.children.append((-1.0,z)) #y.child =[dz/dy=-1, z]
        return z

    def __truediv__(self, other): #z=x/y, dz/dx=1/y, dz/dy=-x/y^2
        z=Var(self.value/other.value)
        self.children.append((1/other.value,z))  #x.child =[dz/dx=1/y, z]
        other.children.append((-self.value/other.value**2,z)) #y.child =[dz/dy=-x/y^2, z]
        return z


    def __pow__(self, other): #z=x^y, dz/dx= yx^(y-1), dz/dy= x^y ln(x)
        
        a,b=1,1

        #CAUTION: This will only work if other.value is an integer.  What if 0.5^(4.2) ?
        for i in range(other.value-1):
          a*=self.value  #this is x^(y-1)
          b*=self.value 
        b*=self.value  #this is x^y

        z=Var(b)

        self.children.append((other.value*a,z))   #x.child = [dz/dx=yx^(y-1),z]
        other.children.append((b*math.log(self.value),z)) #y.child = [dz/dy=x^y ln x,z]

    
        return z

def sin(x):
    z = Var(math.sin(x.value))
    x.children.append((math.cos(x.value), z))
    return z




In [ ]:
# Tests

#Var(1)**Var(1)


Var(1) + Var(1) / Var(1) - Var(1)**Var(1)


#Forward mode computation

In [ ]:


x=Var(0.5)
y=Var(4.2)

a=x*y
b=sin(x)
z=a+b

def printGradValue():
  print('--grad_value--')
  print(x.grad_value)
  print(y.grad_value)
  print(a.grad_value)
  print(b.grad_value)


#print(x.children)



print(f'\n{x.children[0][0]},{x.children[0][1]}  <-- This is first child of x which is a. da/dx=4.2 with  a=4.2*0.5=2.1')
print(f'{x.children[1][0]:.2f},{x.children[1][1]}  <-- This is second child of x which is b. db/dx=cos(0.5)=0.87 with b=sin(0.5)=0.479\n')

print(f'{y.children[0][0]},{y.children[0][1]} <-- y only has 1 child which is a. da/dy=0.5 with  a=4.2*0.5=2.1\n')

print(f'{a.children[0][0]},{a.children[0][1]} <-- a only has 1 child which is z. dz/da=1 with  z=a*b=2.57..\n')
print(f'{b.children[0][0]},{b.children[0][1]} <-- b too only has 1 child which is z. dz/db=1 with  z=a*b=2.57..\n')



4.2,2.1  <-- This is first child of x which is a. da/dx=4.2 with  a=4.2*0.5=2.1
0.88,0.479425538604203  <-- This is second child of x which is b. db/dx=cos(0.5)=0.87 with b=sin(0.5)=0.479

0.5,2.1 <-- y only has 1 child which is a. da/dy=0.5 with  a=4.2*0.5=2.1

1.0,2.579425538604203 <-- a only has 1 child which is z. dz/da=1 with  z=a*b=2.57..

1.0,2.579425538604203 <-- b too only has 1 child which is z. dz/db=1 with  z=a*b=2.57..



#Reverse mode computation

So far we have done forward computing as we go. But we haven't computed $\frac{\partial z}{\partial x}$ and $\frac{\partial z}{\partial y}$ 

In [ ]:
printGradValue()# This should be None

z.grad_value=1 #Seeding 

print('z: ',z.value)

print("dz/dx: ",x.grad())
print("dz/dy: ",y.grad())
print("dz/da: ",a.grad())
print("dz/db: ",b.grad())

printGradValue() #Only after seeding this has value



#Test your computed values
assert abs(z.value - 2.579425538604203) <= 1e-15
assert abs(x.grad() - (y.value + math.cos(x.value))) <= 1e-15
assert abs(y.grad() - x.value) <= 1e-15

--grad_value--
None
None
None
None
z:  2.579425538604203
dz/dx:  5.077582561890373
dz/dy:  0.5
dz/da:  1.0
dz/db:  1.0
--grad_value--
5.077582561890373
0.5
1.0
1.0
